In [ ]:
import pandas as pd
import os
import numpy as np

import re
# import json
import matplotlib.pyplot as plt

import random
import shutil

In [ ]:
df_data = pd.read_json("D:/NAACL/348.json" , lines = True)

In [ ]:
list(range(5))

In [ ]:
# Second json with data
df_data2 = pd.read_json("D:/NAACL/core_all_cyr.jsonl" , lines = True)

In [ ]:
df_full = df_data.append(pd.DataFrame(data = df_data2), ignore_index=True)
df_full = df_full.drop_duplicates(subset=["coreId"],keep = "first")

In [ ]:
# get the final PDF Core_ids
files = os.listdir("C:/Masterarbeit/Data/manuel_annotated/labelled_text")
# f in files re.sub("Core_ID_","",files)
core_ids= [int(re.sub(".txt","",f)) for f in files]
len(core_ids)

In [ ]:
df_full = df_full[df_full.coreId.isin(core_ids)]

In [ ]:
df_full.reset_index(drop = True , inplace= True)

In [ ]:
df_full

In [ ]:
df_full.info()

# Source

In [ ]:
string = []
for i in df_full.relations:
    try:
        string.append(i[0])
    except:
        string.append("ERROR")

In [ ]:
df_full.relations = string

In [ ]:
string2 = [re.sub("[0-9]", "",x) for x in df_full.relations]

In [ ]:
from collections import Counter

Counter(string2)

## Histogram of year distribution

In [ ]:
vec_year = [df_full.year[df_full.coreId.isin(core_ids)].iloc[i] for i in range(len(core_ids))]
df_year = df_full.year[df_full.coreId.isin(core_ids)]

In [ ]:
df_year.describe()

In [ ]:
from collections import Counter

Counter(vec_year)

In [ ]:
# Number of years
len(list(set(vec_year)))-1

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# gaussian_numbers = np.random.normal(size=10000)
plt.hist(vec_year, bins=10)
plt.title("Gaussian Histogram of publication year")
plt.xlabel("Year")
plt.ylabel("Frequency of papers")
# plt.show()

plt.savefig('publication_year.pdf'  ,  bbox_inches='tight')

# Topics

In [ ]:
topics = []
for topic in list(df_full.topics):
    try:
        if re.match("([a-zA-Z]|[А-Яа-я])",topic[0]):
            top = re.sub("Выпуск \d*. Серия: ","",topic[0])
            top = ''.join([i for i in top if not i.isdigit()])
            top = top.replace("""(""","").replace(""")""","")
            if re.search("(коммуналь)|(город)",top.lower())!=None:
                topics.append("Urban Planning/Infrastructure")
            elif re.search("(эконом)|(економічні)",top.lower())!=None:
                topics.append("Economics")
            elif re.search("(технические науки)|(технологии)|(технічні)",top.lower())!=None:
                topics.append("Technics")
            elif re.search("(education)|(воспитания)",top.lower())!=None:
                topics.append("Education")
            elif re.search("(электроэнергетик)|(светотехник)",top.lower())!=None:
#                 topics.append("(электроэнергетик)|(светотехник)")
                topics.append("Technics")
            else:
                topics.append(top)
        else:
             topics.append("no topic")
             pass
    except:
         topics.append("no topic")
         pass

In [ ]:
from collections import Counter
x =Counter(topics)
x 

In [ ]:
x.most_common()[:10]

# Statistics for labelled reference strings

In [ ]:
# get the final PDF Core_ids
files = os.listdir("C:/Masterarbeit/Data/manuel_annotated/labelled_text")
# f in files re.sub("Core_ID_","",files)
core_ids= [int(re.sub(".txt","",f)) for f in files]
len(core_ids)

In [ ]:
files[0]

In [ ]:
all_ref[20]

In [ ]:
count_ref = []
all_ref =[]
for f in files:
    file = open(os.path.join("C:/Masterarbeit/Data/manuel_annotated/labelled_text",f),"r",encoding="utf-8")
    ref_list = file.read().split("\n\n")
    all_ref+=ref_list
    count_ref.append(len(ref_list))

In [ ]:
## Anzahl manuell gelabelter Referenzen
sum(count_ref)

In [ ]:
from collections import Counter
label_list= []
all_labels = []
for ref in all_ref:
    labels = []
    for t in ref.split(""">"""):
#         print(t)
        if "</" not in t and """<""" in t:
            labels.append(t.split("<")[1])
#     print(labels)    
    label_list.append(labels)
    all_labels += labels
count_all_labels = Counter(all_labels)

In [ ]:
count_all_labels.most_common()

In [ ]:
## Sum of labels
i=0
for count in count_all_labels.most_common():
    i+=count[1]
    
print(i)

In [ ]:
import nltk
ref_len = []
all_ref_clean = []
## Average reference length (in tokens)
for text in all_ref:

    for label in set(all_labels):
        text = re.sub(f"(\<{label}\>)|(\</{label}\>)","",text)
    ref_len.append(len(nltk.word_tokenize(text,preserve_line=True)))
    all_ref_clean.append(text)
sum(ref_len)/len(ref_len)

# Language detection

In [ ]:
import fasttext

In [ ]:
# Paper language detection
path_to_txt = "D:/NAACL/Parsed_Text_15553/TXTs_15553"
paper_lang = []
for core_id in core_ids:
    file = open(os.path.join(path_to_txt,"Core_ID_"+str(core_id)+".pdf.txt"),"r",encoding="utf-8")
    file = file.read().replace("\n","")
#     print(file)
    predictions = fasttext_detection_model.predict(file)
    paper_lang.append(predictions[0][0])
Counter(paper_lang)

In [ ]:
len(all_ref_clean)

In [ ]:
# Reference language detection

path_to_txt = "D:/NAACL/Parsed_Text_15553/TXTs_15553"
ref_lang = []
for ref in all_ref_clean:
    ref = ref.replace("\n","")

    predictions = fasttext_detection_model.predict(ref)
    ref_lang.append(predictions[0][0])
ref_lang_count = Counter(ref_lang)
ref_lang_count.most_common()